In [1]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
%matplotlib inline

import sys
np.set_printoptions(threshold=sys.maxsize)

from sklearn import metrics

In [2]:
def show_full_data(data, row_size=None, column_size=None, col_width=-1):
    """
    Shows all rows and columns instead of showing only some part 
    and hiding other parts for large data.
    """
    with pd.option_context('display.max_rows', row_size, 
                           'display.max_columns', column_size, 
                           'display.max_colwidth', col_width):
        display(data)

In [3]:
df = pd.read_csv('combined_samples/combined_data.csv')
# Eliminate first unnecessary column.
df = df.iloc[:, 1:]
df.head()

,timestamp,conversation,bt_level_avg,bt_level_std,bt_total_devices_around,bt_total_far,bt_total_farther,bt_total_near,bt_total_nearer,wifi_level_avg,...,phone_locked,activity_inference_0,activity_inference_1,activity_inference_2,activity_inference_3,audio_inference_0,audio_inference_1,audio_inference_2,audio_inference_3,STRESSED
0,2013-03-27 04:00:00,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-77.0,...,0.0,599.0,0.0,0.0,0.0,274.0,0.0,185.0,140.0,NaN
1,2013-03-27 04:10:00,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,600.0,0.0,0.0,0.0,557.0,1.0,42.0,0.0,NaN
2,2013-03-27 04:20:00,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-71.5,...,0.0,600.0,0.0,0.0,0.0,457.0,0.0,143.0,0.0,NaN
3,2013-03-27 04:30:00,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,208.0,600.0,0.0,0.0,0.0,564.0,1.0,35.0,0.0,NaN
4,2013-03-27 04:40:00,200.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-62.0,...,600.0,600.0,0.0,0.0,0.0,127.0,94.0,379.0,0.0,NaN


In [4]:
# Total number of instances
print(df[df.STRESSED.notnull()].shape[0])

2347


# Create Dataframe with Same Length Instances

In [5]:
sequence_length = 72

In [6]:
def create_same_length_instances(df, length=72, label='STRESSED'):
    df = df.drop(columns=['timestamp'])
    full_data = pd.DataFrame()
    indexes = list(df[df[label].notnull()].index) # Choose label indexes
    start = 0
    for i in indexes:
        if i - start >= length:
            # If the length of instance is higher than treshold length,
            # It chooses last timestamps (according to given length)
            instance = df.iloc[i-length+1:i+1, :]
            full_data = full_data.append(instance, ignore_index=True, sort=False)
        else:
            # If the length of instance is lower than treshold length,
            # It chooses all timestamps and add zeros to head timestamps until reach given length.
            index_diff = i - start
            instance = df.iloc[i-index_diff+1:i+1, :]
            back_fill = np.empty((length-index_diff, df.shape[1]))
            back_fill.fill(np.nan)
            back_fill = pd.DataFrame(back_fill, columns=df.columns)
            instance = back_fill.append(instance, ignore_index=True, sort=False)
            full_data = full_data.append(instance, ignore_index=True, sort=False)
        start = i
    return full_data

In [7]:
df_same = create_same_length_instances(df, length=sequence_length)

In [8]:
print('Total sample size:', df_same[df_same.STRESSED.notnull()].shape[0], '\n')
print('Each class size:\n' + str(df_same.STRESSED.value_counts()))

Total sample size: 2347 

Each class size:
1.0    1614
0.0     733
Name: STRESSED, dtype: int64


In [9]:
df_same[df_same.STRESSED.notnull()].head()

,conversation,bt_level_avg,bt_level_std,bt_total_devices_around,bt_total_far,bt_total_farther,bt_total_near,bt_total_nearer,wifi_level_avg,wifi_level_std,...,phone_locked,activity_inference_0,activity_inference_1,activity_inference_2,activity_inference_3,audio_inference_0,audio_inference_1,audio_inference_2,audio_inference_3,STRESSED
71,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-61.0,NaN,...,0.0,600.0,0.0,0.0,0.0,595.0,2.0,3.0,0.0,1.0
143,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,600.0,0.0,0.0,0.0,356.0,2.0,242.0,0.0,1.0
215,0.0,-89.0,5.618846,7.0,3.0,3.0,1.0,0.0,-84.0,8.447316,...,600.0,598.0,2.0,0.0,0.0,600.0,0.0,0.0,0.0,1.0
287,569.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,600.0,600.0,0.0,0.0,0.0,49.0,510.0,41.0,0.0,1.0
359,600.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-76.0,20.126268,...,0.0,600.0,0.0,0.0,0.0,40.0,536.0,24.0,0.0,0.0


# Normalize Data

In [10]:
def normalize(df, label_col='STRESSED'):
    features = df_same.drop(columns=label_col)
    features_norm = (features - features.mean(axis=0)) / (features.max(axis=0) - features.min(axis=0))
    df.loc[:, df.columns != label_col] = features_norm
    return df

In [11]:
df_norm = normalize(df_same)
show_full_data(df_norm.head())

,conversation,bt_level_avg,bt_level_std,bt_total_devices_around,bt_total_far,bt_total_farther,bt_total_near,bt_total_nearer,wifi_level_avg,wifi_level_std,wifi_total_devices_around,wifi_total_far,wifi_total_near,wifi_total_nearer,phone_in_dark,phone_charging,phone_locked,activity_inference_0,activity_inference_1,activity_inference_2,activity_inference_3,audio_inference_0,audio_inference_1,audio_inference_2,audio_inference_3,STRESSED
0,0.757211,-0.059292,0.012913,0.135913,0.065572,-0.014352,0.150004,-0.027515,0.050303,-0.045424,-0.047465,-0.085929,0.038959,-0.075923,-0.380118,-0.208148,-0.479851,0.103891,-0.060427,-0.014357,-0.029072,-0.237882,0.385415,-0.147496,-0.000036,NaN
1,0.745545,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.380118,-0.208148,-0.479851,0.103891,-0.060427,-0.014357,-0.029072,-0.081215,0.243748,-0.162496,-0.000036,NaN
2,0.435545,-0.059324,0.057321,0.110913,0.065572,0.041203,0.053230,0.034985,0.025127,-0.041241,-0.047465,-0.067061,0.017220,-0.075923,-0.380118,-0.208148,-0.479851,0.103891,-0.060427,-0.014357,-0.029072,-0.587882,0.557082,0.030838,-0.000036,NaN
3,0.757211,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.091883,-0.069595,0.331846,0.366901,0.038959,-0.075923,-0.380118,-0.208148,-0.479851,-0.371109,0.414573,-0.014357,-0.029072,-0.599548,0.667082,-0.067496,-0.000036,NaN
4,0.147211,-0.059450,-0.108500,-0.014087,0.065572,-0.069908,-0.011286,-0.027515,-0.087937,-0.081701,0.349087,0.423505,0.017220,-0.075923,-0.380118,-0.208148,-0.479851,-0.896109,0.939573,-0.014357,-0.029072,-0.236215,0.273748,-0.037496,-0.000036,NaN


In [12]:
print('Total sample size:', df_norm[df_norm.STRESSED.notnull()].shape[0], '\n')
print('Each class size:\n' + str(df_norm.STRESSED.value_counts()))

Total sample size: 2347 

Each class size:
1.0    1614
0.0     733
Name: STRESSED, dtype: int64


# Filling Missing Values

In [13]:
def fill_nulls(df, label_col='STRESSED'):
    df.loc[:, df.columns != label_col] = df.loc[:, df.columns != label_col].fillna(0)
    return df

In [14]:
df_filled = fill_nulls(df_norm)
show_full_data(df_filled.head())

,conversation,bt_level_avg,bt_level_std,bt_total_devices_around,bt_total_far,bt_total_farther,bt_total_near,bt_total_nearer,wifi_level_avg,wifi_level_std,wifi_total_devices_around,wifi_total_far,wifi_total_near,wifi_total_nearer,phone_in_dark,phone_charging,phone_locked,activity_inference_0,activity_inference_1,activity_inference_2,activity_inference_3,audio_inference_0,audio_inference_1,audio_inference_2,audio_inference_3,STRESSED
0,0.757211,-0.059292,0.012913,0.135913,0.065572,-0.014352,0.150004,-0.027515,0.050303,-0.045424,-0.047465,-0.085929,0.038959,-0.075923,-0.380118,-0.208148,-0.479851,0.103891,-0.060427,-0.014357,-0.029072,-0.237882,0.385415,-0.147496,-0.000036,NaN
1,0.745545,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.380118,-0.208148,-0.479851,0.103891,-0.060427,-0.014357,-0.029072,-0.081215,0.243748,-0.162496,-0.000036,NaN
2,0.435545,-0.059324,0.057321,0.110913,0.065572,0.041203,0.053230,0.034985,0.025127,-0.041241,-0.047465,-0.067061,0.017220,-0.075923,-0.380118,-0.208148,-0.479851,0.103891,-0.060427,-0.014357,-0.029072,-0.587882,0.557082,0.030838,-0.000036,NaN
3,0.757211,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.091883,-0.069595,0.331846,0.366901,0.038959,-0.075923,-0.380118,-0.208148,-0.479851,-0.371109,0.414573,-0.014357,-0.029072,-0.599548,0.667082,-0.067496,-0.000036,NaN
4,0.147211,-0.059450,-0.108500,-0.014087,0.065572,-0.069908,-0.011286,-0.027515,-0.087937,-0.081701,0.349087,0.423505,0.017220,-0.075923,-0.380118,-0.208148,-0.479851,-0.896109,0.939573,-0.014357,-0.029072,-0.236215,0.273748,-0.037496,-0.000036,NaN


In [15]:
df_filled[df_filled['STRESSED'].notnull()].index

Int64Index([    71,    143,    215,    287,    359,    431,    503,    575,
               647,    719,
            ...
            168335, 168407, 168479, 168551, 168623, 168695, 168767, 168839,
            168911, 168983],
           dtype='int64', length=2347)

# Create X and y

## X

In [16]:
def create_instances(df, length=72, label='STRESSED'):
    # Converts dataframe to tensor instances to feed lstm network.
    total_instances = df[df[label].notnull()].shape[0]
    total_features = df[df[label].notnull()].shape[1] - 1
    indexes = list(df[df[label].notnull()].index)
    data = df.drop(columns=[label])
    all_data = np.empty(shape=(total_instances, length, total_features))
    for instance_no, label_index in enumerate(indexes):
        start = label_index - length + 1
        all_data[instance_no] = data.iloc[start:label_index+1, :].values
    return all_data

In [17]:
X = create_instances(df_filled, length=sequence_length)

In [18]:
print('Total number of instances:', X.shape[0])
print("One sample's sequence length:", X.shape[1])
print("Feature size:", X.shape[2])

Total number of instances: 2347
One sample's sequence length: 72
Feature size: 25


## y

In [19]:
y = df_filled.loc[df_filled['STRESSED'].notnull(), 'STRESSED'].values.reshape(2347,1)

In [20]:
y.shape

(2347, 1)

# Create Randomly Chosen Train and Test Data with Specific Size

In [21]:
# all_one_indexes = np.where(y == 1)[0]
# all_one_indexes.shape

In [22]:
# all_zero_indexes = np.where(y == 0)[0]
# all_zero_indexes.shape

In [23]:
# ones = np.random.choice(all_one_indexes, 500, replace=False)
# print(ones.shape)
# print(np.unique(ones).shape)

In [24]:
# zeros = np.random.choice(all_zero_indexes, 500, replace=False)
# print(zeros.shape)
# print(np.unique(zeros).shape)

In [25]:
# train_indexes = np.concatenate((ones, zeros))
# print(train_indexes.shape)
# print(np.unique(train_indexes).shape)

In [26]:
# train_indexes[:50]

In [27]:
# np.random.shuffle(train_indexes)

In [28]:
# train_indexes[:50]

In [29]:
# new_x = X[train_indexes, :, :]
# new_y = y[train_indexes, :]

In [30]:
# print(new_x.shape)
# print(new_y.shape)

In [31]:
# print(np.where(new_y == 1)[0].shape)
# print(np.where(new_y == 0)[0].shape)

In [32]:
# test_indexes = np.array([i for i in range(len(y)) if i not in train_indexes])
# print(test_indexes.shape)

In [33]:
# np.unique(list(set(train_indexes) - set(test_indexes))).shape

In [34]:
# not_used_ones = np.array(list(set(all_one_indexes) - set(ones)))
# print(not_used_ones.shape)
# print(np.unique(not_used_ones).shape)

In [35]:
# np.unique(list(set(not_used_ones) - set(train_indexes))).shape

In [36]:
# not_used_zeros = np.array(list(set(all_zero_indexes) - set(zeros)))
# print(not_used_zeros.shape)
# print(np.unique(not_used_zeros).shape)

In [37]:
# test_sample_size = min(len(not_used_ones), len(not_used_zeros))
# print(test_sample_size)

In [38]:
# test_ones = np.random.choice(not_used_ones, test_sample_size, replace=False)
# print(test_ones.shape)
# print(np.unique(test_ones).shape)

In [39]:
# test_zeros = np.random.choice(not_used_zeros, test_sample_size, replace=False)
# print(test_zeros.shape)
# print(np.unique(test_zeros).shape)

In [40]:
# test_indexes = np.concatenate((test_ones, test_zeros))

In [41]:
# np.random.shuffle(test_indexes)

In [42]:
# test_x = X[test_indexes, :, :]
# test_y = y[test_indexes, :]

In [43]:
# print(test_x.shape)
# print(test_y.shape)

In [44]:
# print(np.where(test_y == 1)[0].shape)
# print(np.where(test_y == 0)[0].shape)

In [45]:
# np.unique(list(set(train_indexes) - set(test_indexes))).shape

In [46]:
# np.unique(list(set(test_indexes) - set(train_indexes))).shape

In [47]:
def create_random_sets(x, y, train_size=1000, seed=1, balanced_test=False):
    # Randomly chooses created instances and created balanced train test set.
    # If balanced_test==True, it creates balanced test set. 
    # (If False, takes all remaning data as test.)
    random.seed(seed)
    one_class_size = int(train_size / 2)
    all_one_indexes = np.where(y == 1)[0]
    all_zero_indexes = np.where(y == 0)[0]
    ones = np.random.choice(all_one_indexes, one_class_size, replace=False)
    zeros = np.random.choice(all_zero_indexes, one_class_size, replace=False)
    train_indexes = np.concatenate((ones, zeros))
    np.random.shuffle(train_indexes)
    new_x = x[train_indexes, :, :]
    new_y = y[train_indexes, :]
    if balanced_test == False:
        test_indexes = np.array([i for i in range(len(y)) if i not in train_indexes])
        np.random.shuffle(test_indexes)
        test_x = x[test_indexes, :, :]
        test_y = y[test_indexes, :]
    else:
        not_used_ones = np.array(list(set(all_one_indexes) - set(ones)))
        not_used_zeros = np.array(list(set(all_zero_indexes) - set(zeros)))
        test_sample_size = min(len(not_used_ones), len(not_used_zeros))
        test_ones = np.random.choice(not_used_ones, test_sample_size, replace=False)
        test_zeros = np.random.choice(not_used_zeros, test_sample_size, replace=False)
        test_indexes = np.concatenate((test_ones, test_zeros))
        np.random.shuffle(test_indexes)
        test_x = x[test_indexes, :, :]
        test_y = y[test_indexes, :]
    return new_x, new_y, test_x, test_y, train_indexes, test_indexes

In [48]:
X_train, y_train, X_test, y_test, train_indexes, test_indexes = create_random_sets(X, y, train_size=1200, seed=53, balanced_test=True)

In [49]:
print('X_train shape:', X_train.shape)
print('y_train shape:', y_train.shape)
print('X_test shape:', X_test.shape)
print('y_test shape:', y_test.shape)

X_train shape: (1200, 72, 25)
y_train shape: (1200, 1)
X_test shape: (266, 72, 25)
y_test shape: (266, 1)


In [50]:
np.unique(test_indexes).shape

(266,)

In [51]:
# np.sort(test_indexes)

In [52]:
# np.sort(train_indexes)

In [53]:
# np.unique(train_indexes).shape

In [54]:
# len(list(set(train_indexes) - set(test_indexes)))

# Model 

In [55]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Embedding
from keras.layers import LSTM
from keras.optimizers import Adam
from keras.optimizers import Adagrad
from keras.optimizers import SGD
import keras

Using TensorFlow backend.


In [56]:
max_features = 25

In [57]:
model = Sequential()
# model.add(Embedding(25, 10, input_length=sequence_length)) # Does not work.
model.add(LSTM(32, activation='tanh', dropout=0.2, return_sequences=True, input_shape=(sequence_length, 25))) # returns a sequence of vectors of dimension 32
# model.add(LSTM(32, activation='tanh', return_sequences=True))  # returns a sequence of vectors of dimension 32
# model.add(LSTM(32, activation='tanh', return_sequences=True))  # returns a sequence of vectors of dimension 32
model.add(LSTM(32, activation='tanh', dropout=0.2, return_sequences=True))  # returns a sequence of vectors of dimension 32
model.add(Dense(32, activation='tanh'))
model.add(LSTM(32, activation='tanh', dropout=0.2))  # return a single vector of dimension 32
model.add(Dense(32, activation='tanh'))
# model.add(Dense(32))
# model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

In [58]:
optim = Adam(lr=0.001, 
             beta_1=0.9, 
             beta_2=0.999, 
             epsilon=None, 
             decay=0.0, 
             amsgrad=False)

# optim = Adagrad(lr=0.01, 
#                 epsilon=None, 
#                 decay=0.0)

# optim = SGD(lr=0.001, 
#             decay=1e-6, 
#             momentum=0.9, 
#             nesterov=True,
#             clipnorm=1.0)

In [59]:
model.compile(loss='binary_crossentropy',
              optimizer=optim,
              metrics=['accuracy'])

In [60]:
my_callbacks = [
    keras.callbacks.EarlyStopping(monitor='val_loss', 
                                  min_delta=0.00001, 
                                  patience=50, 
                                  verbose=0, 
                                  mode='auto', 
                                  baseline=None, 
                                  restore_best_weights=True),
    
    keras.callbacks.ModelCheckpoint('model_weights/weights.{epoch:02d}-{val_acc:.2f}.hdf5',
                                    monitor='val_loss', 
                                    verbose=0, 
                                    save_best_only=True, 
                                    save_weights_only=False, 
                                    mode='auto', 
                                    period=1),
    
    keras.callbacks.TensorBoard(log_dir='model_weights/tensorboard/', 
                                histogram_freq=0, 
                                batch_size=32, 
                                write_graph=True, 
                                write_grads=False, 
                                write_images=True, 
                                embeddings_freq=0, 
                                embeddings_layer_names=None, 
                                embeddings_metadata=None, 
                                embeddings_data=None, 
                                update_freq='epoch'), 
    
#     keras.callbacks.ReduceLROnPlateau(monitor='val_loss', 
#                                       factor=0.2, 
#                                       patience=10, 
#                                       min_lr=0.0001), 
    
    keras.callbacks.CSVLogger('model_weights/csv_logger/train_log.csv', separator=',', append=False)
]

In [61]:
model_history = model.fit(X_train, y_train, batch_size=X_train.shape[0], epochs=500, validation_split=0.2, shuffle=False, callbacks=my_callbacks)

Train on 960 samples, validate on 240 samples
Epoch 1/500
960/960 [==============================] - 2s 2ms/step - loss: 0.6939 - acc: 0.4844 - val_loss: 0.6949 - val_acc: 0.4333
Epoch 2/500
960/960 [==============================] - 1s 577us/step - loss: 0.6930 - acc: 0.5010 - val_loss: 0.6950 - val_acc: 0.4583
Epoch 3/500
960/960 [==============================] - 1s 592us/step - loss: 0.6929 - acc: 0.4979 - val_loss: 0.6951 - val_acc: 0.4667
Epoch 4/500
960/960 [==============================] - 1s 638us/step - loss: 0.6927 - acc: 0.5042 - val_loss: 0.6953 - val_acc: 0.4667
Epoch 5/500
960/960 [==============================] - 1s 624us/step - loss: 0.6926 - acc: 0.5135 - val_loss: 0.6952 - val_acc: 0.4667
Epoch 6/500
960/960 [==============================] - 1s 608us/step - loss: 0.6920 - acc: 0.5063 - val_loss: 0.6949 - val_acc: 0.4750
Epoch 7/500
960/960 [==============================] - 1s 524us/step - loss: 0.6914 - acc: 0.5167 - val_loss: 0.6946 - val_acc: 0.4958
Epoch 8/500

In [62]:
score = model.evaluate(X_test, y_test)

266/266 [==============================] - 1s 6ms/step


In [63]:
print(score)

[0.679858058018792, 0.5789473684210527]


In [64]:
model_history.history

{'val_loss': [0.6926618218421936,
  0.6919943690299988,
  0.6915327310562134,
  0.6910162568092346,
  0.6906114816665649,
  0.690345048904419,
  0.6902182698249817,
  0.6900853514671326,
  0.6900063753128052,
  0.689851701259613,
  0.6895632147789001,
  0.6892606616020203,
  0.688995361328125,
  0.6887547969818115,
  0.6885337233543396,
  0.6884459257125854,
  0.6884072422981262,
  0.6885127425193787,
  0.6884085536003113,
  0.6885955333709717,
  0.6891963481903076,
  0.6896704435348511,
  0.6900292634963989],
 'val_acc': [0.512499988079071,
  0.5083333253860474,
  0.5583333373069763,
  0.5249999761581421,
  0.5208333134651184,
  0.5166666507720947,
  0.5458333492279053,
  0.550000011920929,
  0.550000011920929,
  0.5458333492279053,
  0.5583333373069763,
  0.550000011920929,
  0.5375000238418579,
  0.5291666388511658,
  0.5249999761581421,
  0.5166666507720947,
  0.5,
  0.4958333373069763,
  0.5083333253860474,
  0.5166666507720947,
  0.5208333134651184,
  0.5166666507720947,
  0.5249

In [65]:
np.concatenate((np.around(model.predict(X_test), decimals=2), y_test), axis=1)

array([[0.49000001, 1.        ],
       [0.51999998, 1.        ],
       [0.51999998, 1.        ],
       [0.49000001, 1.        ],
       [0.47999999, 1.        ],
       [0.49000001, 1.        ],
       [0.5       , 1.        ],
       [0.47999999, 1.        ],
       [0.49000001, 0.        ],
       [0.5       , 1.        ],
       [0.49000001, 0.        ],
       [0.49000001, 1.        ],
       [0.50999999, 0.        ],
       [0.49000001, 0.        ],
       [0.5       , 0.        ],
       [0.49000001, 1.        ],
       [0.49000001, 0.        ],
       [0.54000002, 1.        ],
       [0.47999999, 1.        ],
       [0.47      , 0.        ],
       [0.5       , 1.        ],
       [0.51999998, 0.        ],
       [0.5       , 0.        ],
       [0.52999997, 0.        ],
       [0.5       , 0.        ],
       [0.5       , 0.        ],
       [0.5       , 1.        ],
       [0.52999997, 0.        ],
       [0.54000002, 1.        ],
       [0.5       , 0.        ],
       [0.

In [66]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Embedding
from keras.layers import Conv1D, GlobalAveragePooling1D, MaxPooling1D

In [67]:
model = Sequential()
model.add(Conv1D(64, 3, activation='relu', input_shape=(sequence_length, 25)))
model.add(Conv1D(64, 3, activation='relu'))
model.add(MaxPooling1D(3))
model.add(Conv1D(128, 3, activation='relu'))
model.add(Conv1D(128, 3, activation='relu'))
model.add(GlobalAveragePooling1D())
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [68]:
model.fit(X_train, y_train, batch_size=X_train.shape[0], epochs=100, validation_split=0.2, shuffle=True, callbacks=my_callbacks)
# model.fit(X_train, y_train, batch_size=16, epochs=50)
score = model.evaluate(X_test, y_test)

Train on 960 samples, validate on 240 samples
Epoch 1/100
960/960 [==============================] - 1s 1ms/step - loss: 0.6927 - acc: 0.5115 - val_loss: 0.6888 - val_acc: 0.5292
Epoch 2/100
960/960 [==============================] - 0s 32us/step - loss: 0.6895 - acc: 0.5354 - val_loss: 0.6819 - val_acc: 0.5458
Epoch 3/100
960/960 [==============================] - 0s 28us/step - loss: 0.6876 - acc: 0.5521 - val_loss: 0.6850 - val_acc: 0.5167
Epoch 4/100
960/960 [==============================] - 0s 30us/step - loss: 0.6867 - acc: 0.5521 - val_loss: 0.7160 - val_acc: 0.4958
Epoch 5/100
960/960 [==============================] - 0s 31us/step - loss: 0.7280 - acc: 0.5010 - val_loss: 0.6809 - val_acc: 0.5458
Epoch 6/100
960/960 [==============================] - 0s 32us/step - loss: 0.6809 - acc: 0.5656 - val_loss: 0.6773 - val_acc: 0.5542
Epoch 7/100
960/960 [==============================] - 0s 30us/step - loss: 0.6797 - acc: 0.5615 - val_loss: 0.6752 - val_acc: 0.5583
Epoch 8/100
960/9

In [69]:
score

[0.6989397124240273, 0.548872180899283]